<a href="https://colab.research.google.com/github/POde97/Cell-IDpy/blob/main/CellIDExampleGit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q scanpy
!pip install -q mapply
!git clone https://github.com/POde97/Cell-IDpy.git

     |████████████████████████████████| 2.0 MB 4.3 MB/s 
     |████████████████████████████████| 96 kB 4.5 MB/s 
     |████████████████████████████████| 88 kB 7.5 MB/s 
     |████████████████████████████████| 11.2 MB 6.2 MB/s 
     |████████████████████████████████| 965 kB 66.8 MB/s 
     |████████████████████████████████| 1.1 MB 55.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 110 kB 31.6 MB/s 
     |████████████████████████████████| 115 kB 55.5 MB/s 
Cloning into 'Cell-IDpy'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 85 (delta 34), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [2]:
cd Cell-IDpy

/content/Cell-IDpy


In [3]:
from CellID import*
from HyperG import* 
from ProteinCoding import*

In [4]:
url="https://panglaodb.se/markers/PanglaoDB_markers_27_Mar_2020.tsv.gz"
c=pd.read_csv(url, sep='\t', header=0)

c = c[c["species"].isin(["Hs","Mm Hs"])]
c = c[c["organ"]=="Pancreas"]
c = pd.DataFrame(c.groupby('cell type')['official gene symbol'].apply(list))
c.columns = ["signature"]


In [5]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=10rHN_60c2NJvVh8HXcY7P6jNBS9ptp5E' -O "Baron.h5ad"

--2022-10-28 15:13:58--  https://docs.google.com/uc?export=download&id=10rHN_60c2NJvVh8HXcY7P6jNBS9ptp5E
Resolving docs.google.com (docs.google.com)... 74.125.23.100, 74.125.23.102, 74.125.23.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.23.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-8c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/no2prhkjrnkv4oun7f5h2isc3acf0pgi/1666970025000/15472246493323528491/*/10rHN_60c2NJvVh8HXcY7P6jNBS9ptp5E?e=download&uuid=815b7efb-db68-4603-bef0-732f9ccd212f [following]
--2022-10-28 15:14:02--  https://doc-0k-8c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/no2prhkjrnkv4oun7f5h2isc3acf0pgi/1666970025000/15472246493323528491/*/10rHN_60c2NJvVh8HXcY7P6jNBS9ptp5E?e=download&uuid=815b7efb-db68-4603-bef0-732f9ccd212f
Resolving doc-0k-8c-docs.googleusercontent.com (doc-0k-8c-docs.googleusercontent.com)... 108.177.125.132, 2404:6800:400

In [6]:
adata = sc.read("Baron.h5ad")
adata = adata[:,adata.var_names.isin(HgProteinCoding())]
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.scale(adata)


/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/usr/local/lib/python3.7/dist-packages/scanpy/preprocessing/_normalization.py:170: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  view_to_actual(adata)


In [7]:
cID = CellID(adata)

Computing Fuzzy Matrix
Computing SVD
Computing coordinates
Computing Cell-Genes Distances
Build signature with TOP 200 closest genes
Storing MCA in adata object


In [8]:
HGT = Hypergeom(cID.ad,c)

  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
from sklearn.metrics import accuracy_score
adata1 = cID.ad[cID.ad.obs["gs_prediction"]!="unassigned"]
accuracy_score(list(adata1.obs["cell.type"]), list(adata1.obs["gs_prediction"]))

0.9518487858719646